In [1]:
import pandas as pd
import os
import re
import unicodedata
from rapidfuzz import fuzz, process as fuzzprocess
import numpy as np
import os
import unicodedata
import pandas as pd

In [2]:
_re_non_alnum = re.compile(r"[^0-9a-z]+")
_re_multi_unders = re.compile(r"_+")

def normalizar_texto(texto: str) -> str:
    """Normaliza texto : sin acentos, minúsculas, underscores limpios."""
    if not isinstance(texto, str):
        return texto

    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join(c for c in texto if not unicodedata.combining(c))
    texto = texto.lower()

    texto = _re_non_alnum.sub("_", texto)
    texto = _re_multi_unders.sub("_", texto)
    return texto.strip("_")

In [3]:
df_vehiculos = pd.read_csv("vehiculos_extraidos.csv")

C:\Users\hiros\AppData\Local\Temp\ipykernel_23328\2481167036.py:1: DtypeWarning: Columns (2,4,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehiculos = pd.read_csv("vehiculos_extraidos.csv")


In [5]:
df_vehiculos.columns

Index(['REGION', 'FOLIO', 'TIPO_SERVICIO', 'PPU', 'FECHA_INGRESO_RNT', 'MARCA',
       'MODELO', 'ANO_FABRICACION', 'CAPACIDAD', 'año_archivo', 'mes_archivo',
       'archivo', 'hoja', 'ANOFABRICACION', 'TIPOSERVICIO',
       'FECHA_INGRESO_SERVICIO', ' TIPO_SERVICIO', 'FECHA _INGRESO_RNT',
       'Tipo_Servicio', 'TIPO_SERVICCIO', 'FECHA_INGRESO_RNTTT',
       'FECHA_INGRESO', 'FECHA INGRESO RNT', 'TIPO_SERVCIO',
       'TIPO_COMBUSTIBLE', 'FECHA_INGRESO_RNT.1', 'TIPO_COMBUSTIBLE ',
       'COMBUSTIBLE', 'FECHA_INGRESO_RNTT', 'TIPO SERVICIO', 'LINEA',
       'AÑO FABRICACION', 'AÑOFABRICACION'],
      dtype='object')

In [6]:

df_vehiculos = df_vehiculos.copy()
df_vehiculos.columns = [normalizar_texto(c) for c in df_vehiculos.columns]


In [7]:
column_groups = {
    "tipo_servicio": [
        "tipo_servicio", "tiposervicio", "tipo_serviccio",
        "tipo_servcio", "tipo_servicio_", "tipo_servicio_1",
        "tipo_servicio1"
    ],
    "fecha_ingreso_rnt": [
        "fecha_ingreso_rnt", "fecha_ingreso_rntt", "fecha_ingreso_rnttt",
        "fecha_ingreso_rnt_1", "fecha_ingreso_rnt1",
        "fecha_ingreso_rnt_", "fecha_ingreso_rnt_"
    ],
    "combustible": [
        "combustible", "tipo_combustible", "tipo_combustible_",
        "tipo_combustible1"
    ],
    "fecha_ingreso": [
        "fecha_ingreso", "fecha_ingreso_servicio", "fecha_ingreso1"
    ],
    "ano_fabricacion": [
        "ano_fabricacion", "anofabricacion", "año_fabricacion",
        "ano_fabricacion_", "ano_fabricacion1"
    ]
}


In [8]:
def unificar_columnas(df, column_groups):
    df = df.copy()

    for final_col, variantes in column_groups.items():
        # filtramos solo variantes que existan de verdad
        existentes = [c for c in variantes if c in df.columns]

        if not existentes:
            continue

        # MÉTODO LIVIANO: tomar el primer no nulo por fila
        df[final_col] = df[existentes].apply(
            lambda row: next((x for x in row if pd.notna(x) and x != ""), None),
            axis=1
        )

        # eliminar duplicadas
        a_eliminar = [c for c in existentes if c != final_col]
        df = df.drop(columns=a_eliminar)

    return df



In [9]:
vehiculos_df = unificar_columnas(df_vehiculos, column_groups)


In [18]:
vehiculos_df.columns

Index(['folio', 'region', 'ppu', 'linea', 'marca', 'modelo', 'capacidad',
       'tipo_servicio', 'tipo_servicio', 'tipo_servicio', 'tipo_servicio',
       'combustible', 'fecha_ingreso', 'fecha_ingreso_rnt',
       'fecha_ingreso_rnt', 'fecha_ingreso_rnt', 'ano_fabricacion',
       'ano_fabricacion', 'ano_archivo', 'mes_archivo', 'archivo', 'hoja',
       'ano_fabricacion', 'ano_fabricacion'],
      dtype='object')

### Consolidar columnas

In [21]:
# Eliminar columnas duplicadas (mantener la primera aparición)
vehiculos_df = vehiculos_df.loc[:, ~vehiculos_df.columns.duplicated()]
vehiculos_df.columns

Index(['folio', 'region', 'ppu', 'marca', 'modelo', 'ano_fabricacion',
       'capacidad', 'tipo_servicio', 'combustible', 'fecha_ingreso',
       'fecha_ingreso_rnt', 'ano_archivo', 'mes_archivo'],
      dtype='object')

In [22]:
# Ordenar dataframe
column_order = [
    "folio",
    "region",
    "ppu",
    "marca",
    "modelo",
    "ano_fabricacion",
    "capacidad",
    "tipo_servicio",
    "combustible",
    "fecha_ingreso",
    "fecha_ingreso_rnt",
    "ano_archivo",
    "mes_archivo"
]

vehiculos_df = vehiculos_df[column_order]

vehiculos_df.head()

,folio,region,ppu,marca,modelo,ano_fabricacion,capacidad,tipo_servicio,combustible,fecha_ingreso,fecha_ingreso_rnt,ano_archivo,mes_archivo
0,10,1.0,RW2882,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,TAXI COLECTIVO RURAL,None,None,2007-06-26,2021-01-01,abril
1,12,1.0,BCRH67,HYUNDAI,ACCENT 1.6,2015.0,NaN,TAXI COLECTIVO RURAL,None,None,2019-10-07,2021-01-01,abril
2,18,1.0,DWCY37,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,TAXI COLECTIVO RURAL,None,None,2012-05-15,2021-01-01,abril
3,19,1.0,DGFX88,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,TAXI COLECTIVO RURAL,None,None,2011-08-30,2021-01-01,abril
4,24,1.0,FPCP93,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,TAXI COLECTIVO RURAL,None,None,2013-05-03,2021-01-01,abril


In [23]:
vehiculos_df.isnull().sum()

folio                      0
region                     1
ppu                        0
marca                   1377
modelo                  1866
ano_fabricacion         1377
capacidad            2719799
tipo_servicio              0
combustible          8125686
fecha_ingreso        9940664
fecha_ingreso_rnt    2316732
ano_archivo                0
mes_archivo                0
dtype: int64

In [24]:
vehiculos_df

,folio,region,ppu,marca,modelo,ano_fabricacion,capacidad,tipo_servicio,combustible,fecha_ingreso,fecha_ingreso_rnt,ano_archivo,mes_archivo
0,10,1.0,RW2882,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,TAXI COLECTIVO RURAL,None,None,2007-06-26,2021-01-01,abril
1,12,1.0,BCRH67,HYUNDAI,ACCENT 1.6,2015.0,NaN,TAXI COLECTIVO RURAL,None,None,2019-10-07,2021-01-01,abril
2,18,1.0,DWCY37,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,TAXI COLECTIVO RURAL,None,None,2012-05-15,2021-01-01,abril
3,19,1.0,DGFX88,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,TAXI COLECTIVO RURAL,None,None,2011-08-30,2021-01-01,abril
4,24,1.0,FPCP93,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,TAXI COLECTIVO RURAL,None,None,2013-05-03,2021-01-01,abril
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480787,500700,16.0,DKWJ44,SCANIA,K380B,2012.0,44.0,BUS RURAL CORRIENTE,None,None,None,2019-01-01,diciembre
10480788,500936,16.0,BXDK27,MERCEDES BENZ,0500 RS,2009.0,NaN,BUS RURAL CORRIENTE,None,None,None,2019-01-01,diciembre
10480789,500825,16.0,KXBT57,MERCEDES BENZ,OF 1724,2019.0,46.0,BUS RURAL CORRIENTE,None,None,None,2019-01-01,diciembre
10480790,400080,16.0,WK7479,MERCEDES BENZ,LO 914 42,2007.0,NaN,BUS URBANO CORRIENTE,None,None,None,2019-01-01,diciembre


In [25]:
vehiculos_df.to_parquet("../utils/vehiculos_limpios.parquet", index=False)